**深度学习**称作端到端的学习, End-To-End Machine Leaning, **从原始数据中获得目标结果**

In [4]:
import numpy as np
def mean_square_error(y, t):
    return 0.5 * np.sum((y - t) ** 2)

t = [0,   0,    1,   0,   1]
y = [0.1, 0.05, 0.6, 0.1, 0.095 ]
np.sum(np.array(y))

0.945